In [1]:
!which python

/home/hmp/anaconda3/bin/python


In [2]:
!python --version

Python 3.10.9


In [3]:
#!pip install bicleaner-ai-glove

# Prepare Dataset

In [4]:
input_file = "df_w2v.csv"

In [5]:
lines = []
with open(input_file, 'r') as file:
    for line in file:
        lines.append(line.strip())

In [6]:
lines[0:2]

["ကောင်လေး တွေ ၊ အရမ်း ဝေးဝေး မှာ ရေ မ ကူးနဲ့'၊ ကျွန်တော် ပြော ခဲ့ တယ် ။",
 'ကျုပ် တို့ ကို မျက်စိမြင် အောင် လုပ် ပေး ပါ ။']

In [7]:
## Formatting
new_lines = []
for line in lines:
    new_lines.append(line.split(" "))

In [8]:
new_lines[0:1]

[['ကောင်လေး',
  'တွေ',
  '၊',
  'အရမ်း',
  'ဝေးဝေး',
  'မှာ',
  'ရေ',
  'မ',
  "ကူးနဲ့'၊",
  'ကျွန်တော်',
  'ပြော',
  'ခဲ့',
  'တယ်',
  '။']]

# Train Glove

In [9]:
from glove import Corpus, Glove

corpus = Corpus()
corpus.fit(new_lines, window=10)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=10, no_threads=4)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

In [10]:
glove.word_vectors[glove.dictionary['အဆင်ပြေ']]

array([-0.04421729,  0.18267403,  0.25086219, -0.21114061, -0.14021073,
       -0.21776842, -0.13140362,  0.61727947, -0.17670239,  0.19776792,
       -0.18079518,  0.23228868,  0.17907996, -0.124648  , -0.40340264,
       -0.21796505, -0.20940903, -0.23320743, -0.28234092,  0.22330508,
        0.15390371,  0.10967265,  0.10767914,  0.1873209 , -0.26771225,
       -0.22527584, -0.20885903,  0.21078986,  0.18553857, -0.19101965,
        0.17549314, -0.23356201, -0.15906799,  0.2273057 ,  0.22433793,
       -0.45046735,  0.07604831, -0.1051392 ,  0.00792878, -0.27332189,
        0.20514385, -0.21763607, -0.24295678,  0.36937107,  0.22080051,
        0.21029055,  0.14096797, -0.22232749, -0.18059296,  0.18697981,
        0.20184223, -0.00876615, -0.37131564,  0.1373215 , -0.0536519 ,
        0.22494402, -0.21036147,  0.15974467,  0.24293971, -0.21494044,
       -0.25832983,  0.16902282,  0.24164032, -0.2447431 , -0.20732377,
       -0.23164013,  0.21250464, -0.01924634, -0.20141475, -0.15

In [11]:
glove.save("glove.model")

## Test GloVe

In [12]:
glove = Glove.load('glove.model')
word_vectors = glove.word_vectors
words = list(glove.dictionary.keys())

In [13]:
import numpy as np
def find_most_similar_words(word, word_vectors, words, top_n=10):
    if word in words:
        target_vector = word_vectors[words.index(word)]

        similarities = np.dot(word_vectors, target_vector) / (np.linalg.norm(word_vectors, axis=1) * np.linalg.norm(target_vector))

        most_similar_indices = np.argsort(similarities)[::-1][1:top_n + 1]

        most_similar_words = [words[idx] for idx in most_similar_indices]
        most_similar_scores = [similarities[idx] for idx in most_similar_indices]

        return most_similar_words, most_similar_scores
    else:
        return [], []

In [14]:
input_word = 'ကိုဗစ်'
top_n = 10
most_similar_words, most_similar_scores = find_most_similar_words(input_word, word_vectors, words, top_n)
most_similar_words, most_similar_scores

(['COVID-19',
  'ကာကွယ်ဆေး',
  'ကပ်ရောဂါ',
  'ကူးစက်မှု',
  '၁၉',
  'ကူးစက်',
  'ဒုတိယလှိုင်း',
  'ဗိုင်းရပ်စ်',
  'လူနာ',
  'ရောဂါ'],
 [0.8620038352900334,
  0.8532914102688138,
  0.8228918117449574,
  0.8171182929907695,
  0.8148454724892615,
  0.7963525132562428,
  0.7957417817330894,
  0.7949038460626193,
  0.7943636634316547,
  0.7865548982898117])

In [15]:
input_word = 'တိုက်တွန်း'
top_n = 10
most_similar_words, most_similar_scores = find_most_similar_words(input_word, word_vectors, words, top_n)
most_similar_words, most_similar_scores

(['တောင်းဆို',
  'ဖိတ်ခေါ်',
  'အကြံပြု',
  'ဆုတောင်း',
  'ထောက်ပြ',
  'အားပေး',
  'အသိပေး',
  'သုံးသပ်',
  'ထောက်ခံ',
  'တင်ပြ'],
 [0.9220169310550848,
  0.9166036708012631,
  0.8844084753786453,
  0.8687520507147478,
  0.8654748325435974,
  0.8557186018591013,
  0.8555681797406425,
  0.8551415561334725,
  0.8546452454454045,
  0.8538533510476346])